In [1]:
print('Début du Notebook')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1632138484802_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Début du Notebook

In [43]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- -------
beautifulsoup4             4.8.1
boto                       2.49.0
boto3                      1.18.44
botocore                   1.21.44
jmespath                   0.9.4
lxml                       4.4.2
mysqlclient                1.4.6
nltk                       3.4.5
nose                       1.3.4
numpy                      1.14.5
pip                        21.2.4
py-dateutil                2.2
python-dateutil            2.8.2
python36-sagemaker-pyspark 1.2.6
pytz                       2019.3
PyYAML                     3.11
s3transfer                 0.5.0
setuptools                 58.0.4
six                        1.13.0
soupsieve                  1.9.5
urllib3                    1.26.6
wheel                      0.37.0
windmill                   1.6

# I.Chargment des images

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit
from functools import reduce

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession.builder.appName('Classif-fruits').getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
test_folder = 's3://p8-prevot/data/Test/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
sc.install_pypi_package("boto3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [119]:
import boto3

bucket = 'p8-prevot'

prefix = 'data/Test/'

client = boto3.client('s3')
result = client.list_objects(Bucket=bucket, Prefix=prefix, Delimiter='/')

for o in result.get('CommonPrefixes'):
    print('sub folder : ', o.get('Prefix'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

sub folder :  data/Test/apple_red_2/
sub folder :  data/Test/carrot_1/
sub folder :  data/Test/zucchini_1/

In [21]:
folder = 'data/Test/apple_red_2/'
sub_folder = folder.split('/')[2]
data_path = 's3://{}/{}'.format(bucket, folder)
images_df = ImageSchema.readImages(data_path, recursive=True).withColumn("label", lit(sub_folder))
images_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+
|               image|      label|
+--------------------+-----------+
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
|[s3://p8-prevot/d...|apple_red_2|
+--------------------+-----------+
only showing top 20 rows

In [29]:
images_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = false)
 |    |-- width: integer (nullable = false)
 |    |-- nChannels: integer (nullable = false)
 |    |-- mode: integer (nullable = false)
 |    |-- data: binary (nullable = false)
 |-- label: string (nullable = false)

### Exploration de la structure du dataframe résultant du readImages

In [32]:
first_row = images_df.first()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Emplacement

In [34]:
first_row.asDict()['image']['origin']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://p8-prevot/data/Test/apple_red_2/r0_103.jpg'

Hauteur

In [35]:
first_row.asDict()['image']['height']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

420

Largeur

In [36]:
first_row.asDict()['image']['width']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

447

Nombre de canaux: 3 (=RVB), 2 si en noir et blanc

In [37]:
first_row.asDict()['image']['nChannels']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3

Mode 16 bit d'encodage de la valeur de rouge vert ou bleu

In [38]:
first_row.asDict()['image']['mode']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16

Données de l'image: Affichage de l'image

In [47]:
data_row = first_row.asDict()['image']['data']  # format byte_array

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
sc.uninstall_package("numpy")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Found existing installation: numpy 1.15.0
Uninstalling numpy-1.15.0:
  Successfully uninstalled numpy-1.15.0

In [110]:
sc.install_pypi_package("numpy==1.15.0")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached numpy-1.15.0-cp36-cp36m-manylinux1_x86_64.whl (13.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.14.5
    Not uninstalling numpy at /usr/local/lib64/python3.6/site-packages, outside environment /tmp/1632139472049-0
    Can't uninstall 'numpy'. No files were found to uninstall.

In [107]:
sc.install_pypi_package("pyspark==2.3.2")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Created wheel for pyspark: filename=pyspark-2.3.2-py2.py3-none-any.whl size=212344378 sha256=89b44980bb0fd576aaa12f085ba9e3f0980412b4f6d60af1a82f1dfa3b3f43cd
  Stored in directory: /mnt/var/lib/livy/.cache/pip/wheels/0b/9f/11/1345c7d24df142160582b83b52f652542e8b708ebf73e13743
Successfully built pyspark

In [114]:
sc.install_pypi_package("matplotlib")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached matplotlib-3.3.4-cp36-cp36m-manylinux1_x86_64.whl (11.5 MB)

In [118]:
import numpy as np

img = np.array(data_row).reshape(first_row.asDict()['image']['height'],
                           first_row.asDict()['image']['width'],
                           first_row.asDict()['image']['nChannels'])

img.shape

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(420, 447, 3)

## Création d'un unique dataframe

In [120]:
def image_df(bucket='p8-prevot', prefix = 'data/Test/'):
    
    '''Permet de créer un dataframe spark contenant les images et leurs caractéristiques'''
    client = boto3.client('s3')
    result = client.list_objects(Bucket=bucket, Prefix=prefix, Delimiter='/')

    dataframes = []
    
    for o in result.get('CommonPrefixes'):
        
        folder = o.get('Prefix')
        sub_folder = folder.split('/')[2]
        data_path = 's3://{}/{}'.format(bucket, folder)
        images_df = ImageSchema.readImages(data_path, recursive=True).withColumn("label", lit(sub_folder))
        dataframes.append(images_df)
        
    df = reduce(lambda first, second: first.union(second), dataframes)# repartition dataframe 
    df = df.repartition(200)# split the data-frame
    
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
df = image_df()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Nous conservons la même structure après la fusion

In [126]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = false)
 |    |-- width: integer (nullable = false)
 |    |-- nChannels: integer (nullable = false)
 |    |-- mode: integer (nullable = false)
 |    |-- data: binary (nullable = false)
 |-- label: string (nullable = false)

In [127]:
train, test = df.randomSplit([0.8, 0.2], 42)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# II. Modèle pré-entrâiné

In [131]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [138]:
#sc.install_pypi_package("sparkdl")
#sc.install_pypi_package("keras")
sc.install_pypi_package("tensorflow")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl (14.8 MB)
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30692 sha256=d9522458703def692daa57989904bf380fa82851bac7c4458ad279daf89b867b
  Stored in directory: /mnt/var/lib/livy/.cache/pip/wheels/22/4c/94/0583f60c9c5b6024ed64f290cb2d43b06bb4f75577dc3c93a7
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4847 sha256=304a8b5589013dafe595e598dcb6332e8d4adb9a02efa8aadd1809d40439c3a5
  Stored in directory: /mnt/var/lib/livy/.cache/pip/wheels/93/2a/eb/e58dbcbc963549ee4f065ff80a59f274cc7210b6eab962acdc
  Created wheel for wrapt: filename=wrapt-1.12.1-cp36-cp36m-linux_x86_64.whl size=67392 sha256=e565bbee98fdfa7131c0cfa1e405d8aa9e03e294940e930ee49f6a110079b330
  Stored in directory: /mnt/var/lib/livy/.cache/pip/wheels/32/42/7f/23cae9ff6ef66798d00dc5d659088e57dbba01566f6c60db63
Successfully built clang termcolor wrapt
  Attempting uninstall: six
    Found existing installation: six 1.

In [139]:
from sparkdl import DeepImageFeaturizer

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SystemError: <built-in method __contains__ of dict object at 0x7f2886dcd990> returned a result with an error set
Traceback (most recent call last):
  File "/tmp/1632139472049-0/local/lib/python3.6/site-packages/sparkdl/__init__.py", line 17, in <module>
    from sparkdl.transformers.keras_image import KerasImageFileTransformer
  File "/tmp/1632139472049-0/local/lib/python3.6/site-packages/sparkdl/transformers/keras_image.py", line 16, in <module>
    import keras.backend as K
  File "/tmp/1632139472049-0/local/lib/python3.6/site-packages/keras/__init__.py", line 21, in <module>
    from tensorflow.python import tf2
  File "/tmp/1632139472049-0/local/lib64/python3.6/site-packages/tensorflow/__init__.py", line 41, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File "/tmp/1632139472049-0/local/lib64/python3.6/site-packages/tensorflow/python/__init__.py", line 40, in <module>
    from tensorflow.python.eager import context
  File "/tmp/1632139472049-0/loc

In [129]:
featurizer = DeepImageFeaturizer(inputCol="image",
                                 outputCol="features",
                                 modelName="InceptionV3")# used as a multi class classifier
lr = LogisticRegression(maxIter=5, regParam=0.03, 
                        elasticNetParam=0.5, labelCol="label")# define a pipeline model
sparkdn = Pipeline(stages=[featurizer, lr])
spark_model = sparkdn.fit(train) # start fitting or training

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'DeepImageFeaturizer' is not defined
Traceback (most recent call last):
NameError: name 'DeepImageFeaturizer' is not defined

